<h2> EDA </h2>

In [1]:
import duckdb
import pandas as pd
from pathlib import Path

DB_PATH = Path(r"C:\Users\serrios\01_prueba_analitico_4\op_cobro\database\analytics.duckdb")
con = duckdb.connect(str(DB_PATH), read_only=True)

def q(sql: str) -> pd.DataFrame:
    """Ejecuta SQL y devuelve pandas df (ideal para resultados pequeños/agrupados)."""
    return con.execute(sql).fetchdf()

print("✅ Conectado a:", DB_PATH)

✅ Conectado a: C:\Users\serrios\01_prueba_analitico_4\op_cobro\database\analytics.duckdb


<p> Lista de tablas en silver: </p>

In [2]:
tables = q("""
SELECT table_name
FROM information_schema.tables
WHERE table_schema='silver'
ORDER BY table_name
""")
tables


,table_name
0,canales
1,clientes
2,excedentes
3,gestiones
4,moras
5,tanque_movimiento


<p> Conteo de filas por tabla </p>

In [4]:
table_list = tables["table_name"].tolist()

row_counts = []
for t in table_list:
    n = con.execute(f"SELECT COUNT(*) FROM silver.{t}").fetchone()[0]
    row_counts.append((t, n))

pd.DataFrame(row_counts, columns=["table_name", "rows"]).sort_values("rows", ascending=False)

,table_name,rows
0,canales,79098
2,excedentes,57753
3,gestiones,57753
4,moras,57753
5,tanque_movimiento,57753
1,clientes,56993


In [9]:
def describe_table(schema: str, table: str) -> pd.DataFrame:
    return q(f"DESCRIBE {schema}.{table}")

def sample_table(schema: str, table: str, n: int = 10) -> pd.DataFrame:
    return q(f"SELECT * FROM {schema}.{table} LIMIT {n}")

def row_count(schema: str, table: str) -> int:
    return con.execute(f"SELECT COUNT(*) FROM {schema}.{table}").fetchone()[0]

def get_columns(schema: str, table: str) -> list[str]:
    return q(f"""
        SELECT column_name
        FROM information_schema.columns
        WHERE table_schema='{schema}' AND table_name='{table}'
        ORDER BY ordinal_position
    """)["column_name"].tolist()

def nulls_profile(schema: str, table: str, top: int = 20) -> pd.DataFrame:
    cols = get_columns(schema, table)
    n = row_count(schema, table)

    exprs = [f"SUM(CASE WHEN {c} IS NULL THEN 1 ELSE 0 END) AS {c}" for c in cols]
    sql = f"SELECT {', '.join(exprs)} FROM {schema}.{table}"

    df = con.execute(sql).fetchdf().T.reset_index()
    df.columns = ["column", "nulls"]
    df["null_pct"] = (df["nulls"] / max(n, 1)).round(4)

    return df.sort_values("nulls", ascending=False).head(top)

def numeric_columns(schema: str, table: str) -> list[str]:
    return q(f"""
        SELECT column_name
        FROM information_schema.columns
        WHERE table_schema='{schema}' AND table_name='{table}'
          AND data_type IN ('INTEGER','BIGINT','DOUBLE','REAL','DECIMAL')
        ORDER BY ordinal_position
    """)["column_name"].tolist()

def numeric_summary(schema: str, table: str, cols: list[str] | None = None, limit_cols: int = 25) -> pd.DataFrame:
    if cols is None:
        cols = numeric_columns(schema, table)[:limit_cols]
    if not cols:
        return pd.DataFrame()

    out = []
    for c in cols:
        df = q(f"""
            SELECT
              '{c}' AS col,
              MIN({c}) AS min,
              AVG({c}) AS avg,
              MAX({c}) AS max,
              quantile_cont({c}, 0.50) AS p50,
              quantile_cont({c}, 0.95) AS p95,
              quantile_cont({c}, 0.99) AS p99
            FROM {schema}.{table}
        """)
        out.append(df)
    return pd.concat(out, ignore_index=True)

def top_values(schema: str, table: str, col: str, top: int = 10) -> pd.DataFrame:
    return q(f"""
        SELECT {col} AS value, COUNT(*) AS n
        FROM {schema}.{table}
        GROUP BY {col}
        ORDER BY n DESC
        LIMIT {top}
    """)

def detect_candidate_keys(schema: str, table: str) -> list[str]:
    """
    Heurística: busca nombres comunes de llave (ajústalo si ya sabes las llaves reales).
    """
    cols = get_columns(schema, table)
    candidates = []
    for c in cols:
        lc = c.lower()
        if lc in ("num_doc", "documento", "id_cliente", "customer_id", "cliente_id", "id", "nit", "cedula"):
            candidates.append(c)
    return candidates

def duplicates_by_key(schema: str, table: str, key: str) -> pd.DataFrame:
    return q(f"""
        SELECT
          COUNT(*) AS rows_total,
          COUNT(DISTINCT {key}) AS distinct_keys,
          (COUNT(*) - COUNT(DISTINCT {key})) AS dup_rows
        FROM {schema}.{table}
    """)

def profile_table(schema: str, table: str, sample_n: int = 10):
    print(f"\n==============================")
    print(f"📌 PERFIL: {schema}.{table}")
    print(f"==============================")

    n = row_count(schema, table)
    print("Filas:", n)

    d = describe_table(schema, table)
    print("Columnas:", len(d))
    display(d)

    print("\n🔎 Nulos (Top):")
    display(nulls_profile(schema, table, top=20))

    keys = detect_candidate_keys(schema, table)
    if keys:
        print("\n🔑 Duplicados por llave candidata:")
        for k in keys:
            print(f"- Key: {k}")
            display(duplicates_by_key(schema, table, k))

    nums = numeric_columns(schema, table)
    if nums:
        print("\n📊 Resumen numérico (Top 20 numéricas):")
        display(numeric_summary(schema, table, cols=nums[:20]))

    print(f"\n👀 Muestra ({sample_n} filas):")
    display(sample_table(schema, table, sample_n))

<h2> 1. Tabla Clientes</h2>

In [10]:
profile_table("silver", "clientes", sample_n=10)


📌 PERFIL: silver.clientes
Filas: 56993
Columnas: 4


,column_name,column_type,null,key,default,extra
0,num_doc,BIGINT,YES,None,None,None
1,obl17,BIGINT,YES,None,None,None
2,f_analisis,DATE,YES,None,None,None
3,var_rta,BIGINT,YES,None,None,None



🔎 Nulos (Top):


,column,nulls,null_pct
1,obl17,4.0,0.0001
0,num_doc,0.0,0.0000
2,f_analisis,0.0,0.0000
3,var_rta,0.0,0.0000



🔑 Duplicados por llave candidata:
- Key: num_doc


,rows_total,distinct_keys,dup_rows
0,56993,33393,23600



📊 Resumen numérico (Top 20 numéricas):


,col,min,avg,max,p50,p95,p99
0,num_doc,5370803333,5.022606e+14,999950837362650,4.995488e+14,9.512756e+14,9.906102e+14
1,obl17,-9353256289591860,4.983127e+16,99998485965389600,4.979231e+16,9.500572e+16,9.897855e+16
2,var_rta,0,6.008106e-01,1,1.000000e+00,1.000000e+00,1.000000e+00



👀 Muestra (10 filas):


,num_doc,obl17,f_analisis,var_rta
0,340880955871439,38183456621181000,2024-07-01,1
1,934023988958985,55043511359914800,2024-07-01,1
2,778281690736332,12180258124215600,2024-07-01,1
3,125044981954652,39056266529610800,2024-07-01,1
4,892818338559716,35529070432914100,2024-07-01,1
5,276178103010500,49904192101969500,2024-07-01,1
6,762963213809470,3972895801599150,2024-07-01,1
7,684757276194578,92124963306271600,2024-07-01,1
8,999568414065449,42557096638927900,2024-07-01,1
9,578250490868374,59381137486483100,2024-07-01,1


<h2> 2. Tabla Canales</h2>

In [11]:
profile_table("silver", "canales", sample_n=10)


📌 PERFIL: silver.canales
Filas: 79098
Columnas: 66


,column_name,column_type,null,key,default,extra
0,num_doc,VARCHAR,YES,None,None,None
1,obl17,VARCHAR,YES,None,None,None
2,f_analisis,DATE,YES,None,None,None
3,total_mnt_digital_ult6,DOUBLE,YES,None,None,None
4,max_mnt_digital_ult6,DOUBLE,YES,None,None,None
...,...,...,...,...,...,...
61,trend_mnt_digital_ult6_vs_ult12,DOUBLE,YES,None,None,None
62,trend_mnt_fisico_ult6_vs_ult12,DOUBLE,YES,None,None,None
63,trend_mnt_efectivo_ult6_vs_ult12,DOUBLE,YES,None,None,None
64,trend_mnt_tarjeta_ult6_vs_ult12,DOUBLE,YES,None,None,None



🔎 Nulos (Top):


,column,nulls,null_pct
1,obl17,4.0,0.0001
42,max_cnt_otros_ult12,0.0,0.0000
48,share_mnt_fisico_ult6,0.0,0.0000
47,share_mnt_digital_ult6,0.0,0.0000
46,total_cnt_all_ult12,0.0,0.0000
45,total_mnt_all_ult12,0.0,0.0000
44,total_cnt_all_ult6,0.0,0.0000
43,total_mnt_all_ult6,0.0,0.0000
0,num_doc,0.0,0.0000
49,share_mnt_efectivo_ult6,0.0,0.0000



🔑 Duplicados por llave candidata:
- Key: num_doc


,rows_total,distinct_keys,dup_rows
0,79098,33393,45705



📊 Resumen numérico (Top 20 numéricas):


,col,min,avg,max,p50,p95,p99
0,total_mnt_digital_ult6,0.0,1.211457e+08,6.735721e+10,3.869702e+07,3.644188e+08,1.102425e+09
1,max_mnt_digital_ult6,0.0,4.325100e+07,2.263858e+10,1.397841e+07,1.358734e+08,4.123072e+08
2,total_cnt_digital_ult6,0.0,2.836895e+02,5.512500e+03,1.796667e+02,8.925000e+02,1.444333e+03
3,max_cnt_digital_ult6,0.0,1.034642e+02,1.905000e+03,6.600000e+01,3.230000e+02,5.730000e+02
4,total_mnt_digital_ult12,0.0,2.306627e+08,1.342749e+11,7.850531e+07,7.068559e+08,2.075543e+09
5,max_mnt_digital_ult12,0.0,8.605354e+07,4.523695e+10,2.985749e+07,2.556152e+08,8.206227e+08
6,total_cnt_digital_ult12,0.0,5.181485e+02,1.002517e+04,3.325833e+02,1.569912e+03,2.734333e+03
7,max_cnt_digital_ult12,0.0,1.969753e+02,4.384000e+03,1.280000e+02,6.140000e+02,1.078000e+03
8,total_mnt_fisico_ult6,0.0,1.440019e+08,2.339867e+11,4.100330e+07,4.843267e+08,1.488754e+09
9,max_mnt_fisico_ult6,0.0,5.478988e+07,5.706767e+10,1.824385e+07,1.674906e+08,5.359501e+08



👀 Muestra (10 filas):


,num_doc,obl17,f_analisis,total_mnt_digital_ult6,max_mnt_digital_ult6,total_cnt_digital_ult6,max_cnt_digital_ult6,total_mnt_digital_ult12,max_mnt_digital_ult12,total_cnt_digital_ult12,...,share_mnt_efectivo_ult12,share_mnt_tarjeta_ult12,share_mnt_otros_ult12,index_digitalizacion_ult12,preferencia_efectivo_ult12,trend_mnt_digital_ult6_vs_ult12,trend_mnt_fisico_ult6_vs_ult12,trend_mnt_efectivo_ult6_vs_ult12,trend_mnt_tarjeta_ult6_vs_ult12,trend_mnt_otros_ult6_vs_ult12
0,922349123250112,5.0986314141055144E16,2025-09-01,18773594.4,7822331.0,40.8,17.0,3.731243e+07,16790593.0,64.444444,...,0.76292,0.022835,0.183046,0.507906,0.76292,0.503146,0.416945,0.203044,0.889409,0.882537
1,899156804630643,7.0537365354430584E16,2024-08-01,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
2,215595954901092,8.0593540479356848E16,2024-09-01,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
3,603989110655846,5.8390705796269608E16,2025-11-01,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
4,280191780739508,1.7350482440027582E16,2025-07-01,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
5,280191780739508,8.7153210794941072E16,2025-07-01,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
6,696190250747254,9.489000766652184E16,2025-11-01,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
7,696190250747254,3.9922602401664656E16,2025-11-01,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
8,334467738071662,5.0461476921252664E16,2024-10-01,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
9,429896526713280,7.1563106867017408E16,2024-12-01,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000


<h2> 3. Tabla Excedente</h2>

In [12]:
profile_table("silver", "excedentes", sample_n=10)


📌 PERFIL: silver.excedentes
Filas: 57753
Columnas: 35


,column_name,column_type,null,key,default,extra
0,num_doc,BIGINT,YES,None,None,None
1,obl17,DOUBLE,YES,None,None,None
2,f_analisis,DATE,YES,None,None,None
3,avg_excedente_pago_3m,DOUBLE,YES,None,None,None
4,min_excedente_pago_3m,DOUBLE,YES,None,None,None
5,max_excedente_pago_3m,DOUBLE,YES,None,None,None
6,stddev_excedente_pago_3m,DOUBLE,YES,None,None,None
7,avg_excedente_pago_6m,DOUBLE,YES,None,None,None
8,min_excedente_pago_6m,DOUBLE,YES,None,None,None
9,max_excedente_pago_6m,DOUBLE,YES,None,None,None



🔎 Nulos (Top):


,column,nulls,null_pct
33,max_porc_pago_12m,2479.0,0.0429
32,min_porc_pago_12m,2479.0,0.0429
31,avg_porc_pago_12m,2479.0,0.0429
34,stddev_porc_pago_12m,2470.0,0.0428
29,max_porc_pago_9m,2328.0,0.0403
28,min_porc_pago_9m,2328.0,0.0403
27,avg_porc_pago_9m,2328.0,0.0403
30,stddev_porc_pago_9m,2317.0,0.0401
24,min_porc_pago_6m,1812.0,0.0314
23,avg_porc_pago_6m,1812.0,0.0314



🔑 Duplicados por llave candidata:
- Key: num_doc


,rows_total,distinct_keys,dup_rows
0,57753,33393,24360



📊 Resumen numérico (Top 20 numéricas):


,col,min,avg,max,p50,p95,p99
0,num_doc,5.370803e+09,5.021152e+14,9.999508e+14,4.993825e+14,9.514646e+14,9.906102e+14
1,obl17,-9.353256e+15,4.985394e+16,9.999849e+16,4.982716e+16,9.498879e+16,9.897985e+16
2,avg_excedente_pago_3m,0.000000e+00,1.069441e+06,1.516603e+09,9.814500e+03,3.718178e+06,1.661865e+07
3,min_excedente_pago_3m,0.000000e+00,3.629772e+05,9.281518e+08,0.000000e+00,8.977061e+05,5.674196e+06
4,max_excedente_pago_3m,0.000000e+00,1.963734e+06,2.690457e+09,1.934204e+04,7.006540e+06,3.153635e+07
5,stddev_excedente_pago_3m,0.000000e+00,8.976723e+05,1.016589e+09,4.607510e+03,3.237981e+06,1.413644e+07
6,avg_excedente_pago_6m,0.000000e+00,1.043397e+06,9.677767e+08,1.611415e+04,3.720307e+06,1.587112e+07
7,min_excedente_pago_6m,0.000000e+00,1.892344e+05,4.284120e+08,0.000000e+00,2.936779e+05,2.662071e+06
8,max_excedente_pago_6m,0.000000e+00,2.509084e+06,2.690457e+09,4.155400e+04,9.201007e+06,4.026873e+07
9,stddev_excedente_pago_6m,0.000000e+00,1.022971e+06,9.303752e+08,1.268267e+04,3.782614e+06,1.599077e+07



👀 Muestra (10 filas):


,num_doc,obl17,f_analisis,avg_excedente_pago_3m,min_excedente_pago_3m,max_excedente_pago_3m,stddev_excedente_pago_3m,avg_excedente_pago_6m,min_excedente_pago_6m,max_excedente_pago_6m,...,max_porc_pago_6m,stddev_porc_pago_6m,avg_porc_pago_9m,min_porc_pago_9m,max_porc_pago_9m,stddev_porc_pago_9m,avg_porc_pago_12m,min_porc_pago_12m,max_porc_pago_12m,stddev_porc_pago_12m
0,254901130982146,5.153062e+16,2025-05-01,820067.82,0.0,1450950.12,743745.54,480910.88,0.0,1450950.12,...,600.00,230.53,268.94,16.59,600.00,230.53,268.94,16.59,600.00,230.53
1,577440651576300,6.638449e+16,2024-12-01,0.00,0.0,0.00,0.00,0.00,0.0,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,456460010588947,7.977339e+16,2025-10-01,0.00,0.0,0.00,0.00,54186.17,0.0,108423.00,...,200.14,81.70,166.70,0.00,300.00,86.62,172.74,0.00,399.87,119.07
3,188542145967389,7.839894e+16,2025-07-01,4910.33,0.0,14731.00,8504.95,2946.20,0.0,14731.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,57372135996850,8.505817e+16,2025-01-01,0.00,0.0,0.00,0.00,0.00,0.0,0.00,...,34.87,24.65,17.43,0.00,34.87,24.65,17.43,0.00,34.87,24.65
5,255871960774551,9.626001e+16,2025-10-01,48424.00,0.0,145272.00,83872.83,29054.40,0.0,145272.00,...,199.44,89.22,59.89,0.00,199.44,89.22,59.89,0.00,199.44,89.22
6,491490762194713,4.840273e+16,2025-04-01,10059.33,7544.0,15090.00,4356.69,7543.40,0.0,15090.00,...,300.00,109.54,180.00,0.00,300.00,109.54,180.00,0.00,300.00,109.54
7,969386116569371,9.224844e+16,2025-08-01,153717.59,0.0,231322.34,133125.43,164106.17,0.0,276989.52,...,235.06,91.52,173.87,0.00,288.74,87.28,173.87,0.00,288.74,87.28
8,394687543115373,2.971538e+16,2025-03-01,862204.67,646373.0,1293868.00,373831.41,646653.50,0.0,1293868.00,...,300.17,125.89,175.04,0.00,300.17,125.89,175.04,0.00,300.17,125.89
9,709585790154184,7.792789e+16,2025-02-01,2437934.36,0.0,5729084.36,2958307.73,2437934.36,0.0,5729084.36,...,1188.45,604.60,529.84,0.00,1188.45,604.60,529.84,0.00,1188.45,604.60


<h2> 4. Tabla gestiones</h2>

In [13]:
profile_table("silver", "gestiones", sample_n=10)


📌 PERFIL: silver.gestiones
Filas: 57753
Columnas: 83


,column_name,column_type,null,key,default,extra
0,num_doc,BIGINT,YES,None,None,None
1,obl17,DOUBLE,YES,None,None,None
2,f_analisis,DATE,YES,None,None,None
3,avg_cant_gestiones_3m,DOUBLE,YES,None,None,None
4,min_cant_gestiones_3m,BIGINT,YES,None,None,None
...,...,...,...,...,...,...
78,stddev_promesas_cumplidas_12m,DOUBLE,YES,None,None,None
79,avg_maximo_rank_12m,DOUBLE,YES,None,None,None
80,min_maximo_rank_12m,DOUBLE,YES,None,None,None
81,max_maximo_rank_12m,DOUBLE,YES,None,None,None



🔎 Nulos (Top):


,column,nulls,null_pct
1,obl17,4.0,0.0001
52,min_cant_acuerdo_9m,0.0,0.0000
60,min_maximo_rank_9m,0.0,0.0000
59,avg_maximo_rank_9m,0.0,0.0000
58,stddev_promesas_cumplidas_9m,0.0,0.0000
57,max_promesas_cumplidas_9m,0.0,0.0000
56,min_promesas_cumplidas_9m,0.0,0.0000
55,avg_promesas_cumplidas_9m,0.0,0.0000
54,stddev_cant_acuerdo_9m,0.0,0.0000
53,max_cant_acuerdo_9m,0.0,0.0000



🔑 Duplicados por llave candidata:
- Key: num_doc


,rows_total,distinct_keys,dup_rows
0,57753,33393,24360



📊 Resumen numérico (Top 20 numéricas):


,col,min,avg,max,p50,p95,p99
0,num_doc,5.370803e+09,5.021152e+14,9.999508e+14,4.993825e+14,9.514646e+14,9.906102e+14
1,obl17,-9.353256e+15,4.985394e+16,9.999849e+16,4.982716e+16,9.498879e+16,9.897985e+16
2,avg_cant_gestiones_3m,0.000000e+00,9.501011e-01,1.125000e+02,0.000000e+00,4.600000e+00,1.300000e+01
3,min_cant_gestiones_3m,0.000000e+00,3.588904e-01,5.000000e+01,0.000000e+00,2.000000e+00,4.000000e+00
4,max_cant_gestiones_3m,0.000000e+00,1.991031e+00,2.360000e+02,0.000000e+00,9.000000e+00,3.500000e+01
5,stddev_cant_gestiones_3m,0.000000e+00,7.754615e-01,1.591000e+02,0.000000e+00,3.540000e+00,1.523440e+01
6,avg_cant_rpc_3m,0.000000e+00,3.120842e-02,7.000000e+00,0.000000e+00,2.000000e-01,8.900000e-01
7,min_cant_rpc_3m,0.000000e+00,2.735789e-03,7.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
8,max_cant_rpc_3m,0.000000e+00,9.512926e-02,1.400000e+01,0.000000e+00,1.000000e+00,2.000000e+00
9,stddev_cant_rpc_3m,0.000000e+00,4.595450e-02,5.920000e+00,0.000000e+00,4.100000e-01,1.210000e+00



👀 Muestra (10 filas):


,num_doc,obl17,f_analisis,avg_cant_gestiones_3m,min_cant_gestiones_3m,max_cant_gestiones_3m,stddev_cant_gestiones_3m,avg_cant_rpc_3m,min_cant_rpc_3m,max_cant_rpc_3m,...,max_cant_acuerdo_12m,stddev_cant_acuerdo_12m,avg_promesas_cumplidas_12m,min_promesas_cumplidas_12m,max_promesas_cumplidas_12m,stddev_promesas_cumplidas_12m,avg_maximo_rank_12m,min_maximo_rank_12m,max_maximo_rank_12m,stddev_maximo_rank_12m
0,727242982424814,2.925142e+16,2025-07-01,0.00,0,0,0.00,0.0,0.0,0.0,...,0,0.00,0.00,0,0,0.00,0.00,0.0,0.0,0.00
1,716173206768778,4.113434e+16,2025-09-01,0.00,0,0,0.00,0.0,0.0,0.0,...,0,0.00,0.00,0,0,0.00,0.00,0.0,0.0,0.00
2,806756957406562,3.287371e+15,2025-03-01,0.00,0,0,0.00,0.0,0.0,0.0,...,0,0.00,0.00,0,0,0.00,0.00,0.0,0.0,0.00
3,768668594687788,2.505004e+16,2025-03-01,3.00,0,5,2.16,0.0,0.0,0.0,...,0,0.00,0.00,0,0,0.00,24.50,20.0,26.0,3.00
4,997061301105873,2.214157e+16,2025-10-01,0.33,0,1,0.58,0.0,0.0,0.0,...,0,0.00,0.00,0,0,0.00,26.00,26.0,26.0,0.00
5,271472900653677,3.017755e+16,2025-03-01,0.00,0,0,0.00,0.0,0.0,0.0,...,0,0.00,0.00,0,0,0.00,0.00,0.0,0.0,0.00
6,650900213820559,5.984198e+16,2025-04-01,0.75,0,2,0.96,0.0,0.0,0.0,...,0,0.00,0.00,0,0,0.00,26.00,26.0,26.0,0.00
7,876100045601351,2.466227e+16,2025-10-01,1.67,1,2,0.58,0.0,0.0,0.0,...,0,0.00,0.00,0,0,0.00,25.73,24.0,26.0,0.65
8,248224647392563,1.507496e+16,2025-05-01,0.00,0,0,0.00,0.0,0.0,0.0,...,0,0.00,0.00,0,0,0.00,0.00,0.0,0.0,0.00
9,24683767748209,3.918376e+16,2025-10-01,2.80,1,5,2.05,0.0,0.0,0.0,...,4,1.05,0.08,0,1,0.28,19.56,1.0,26.0,10.06


<h2> 5 Tabla Moras</h2>

In [14]:
profile_table("silver", "moras", sample_n=10)


📌 PERFIL: silver.moras
Filas: 57753
Columnas: 12


,column_name,column_type,null,key,default,extra
0,num_doc,BIGINT,YES,None,None,None
1,obl17,DOUBLE,YES,None,None,None
2,f_analisis,DATE,YES,None,None,None
3,moras_avg_mora_3m,DOUBLE,YES,None,None,None
4,moras_min_mora_3m,BIGINT,YES,None,None,None
5,moras_max_mora_3m,BIGINT,YES,None,None,None
6,moras_stddev_mora_3m,DOUBLE,YES,None,None,None
7,moras_avg_mora_6m,DOUBLE,YES,None,None,None
8,moras_min_mora_6m,BIGINT,YES,None,None,None
9,moras_max_mora_6m,BIGINT,YES,None,None,None



🔎 Nulos (Top):


,column,nulls,null_pct
1,obl17,4.0,0.0001
0,num_doc,0.0,0.0000
2,f_analisis,0.0,0.0000
3,moras_avg_mora_3m,0.0,0.0000
4,moras_min_mora_3m,0.0,0.0000
5,moras_max_mora_3m,0.0,0.0000
6,moras_stddev_mora_3m,0.0,0.0000
7,moras_avg_mora_6m,0.0,0.0000
8,moras_min_mora_6m,0.0,0.0000
9,moras_max_mora_6m,0.0,0.0000



🔑 Duplicados por llave candidata:
- Key: num_doc


,rows_total,distinct_keys,dup_rows
0,57753,33393,24360



📊 Resumen numérico (Top 20 numéricas):


,col,min,avg,max,p50,p95,p99
0,num_doc,5.370803e+09,5.021152e+14,9.999508e+14,4.993825e+14,9.514646e+14,9.906102e+14
1,obl17,-9.353256e+15,4.985394e+16,9.999849e+16,4.982716e+16,9.498879e+16,9.897985e+16
2,moras_avg_mora_3m,0.000000e+00,5.115937e+00,3.836700e+02,2.000000e+00,1.933000e+01,3.300000e+01
3,moras_min_mora_3m,0.000000e+00,2.394871e+00,6.000000e+01,0.000000e+00,1.400000e+01,2.500000e+01
4,moras_max_mora_3m,0.000000e+00,8.194795e+00,5.840000e+02,4.000000e+00,2.800000e+01,5.200000e+01
5,moras_stddev_mora_3m,0.000000e+00,3.207467e+00,3.315200e+02,1.150000e+00,1.270000e+01,2.358000e+01
6,moras_avg_mora_6m,0.000000e+00,4.953074e+00,5.018300e+02,2.000000e+00,1.900000e+01,3.560000e+01
7,moras_min_mora_6m,0.000000e+00,1.675134e+00,6.000000e+01,0.000000e+00,1.100000e+01,2.100000e+01
8,moras_max_mora_6m,0.000000e+00,9.636763e+00,1.014000e+03,5.000000e+00,3.200000e+01,7.000000e+01
9,moras_stddev_mora_6m,0.000000e+00,3.578184e+00,5.419300e+02,1.470000e+00,1.320000e+01,2.771320e+01



👀 Muestra (10 filas):


,num_doc,obl17,f_analisis,moras_avg_mora_3m,moras_min_mora_3m,moras_max_mora_3m,moras_stddev_mora_3m,moras_avg_mora_6m,moras_min_mora_6m,moras_max_mora_6m,moras_stddev_mora_6m,moras_avg_mora_9m
0,35731704851088,4.049924e+15,2025-04-01,1.00,0,3,1.73,0.75,0,3,1.50,0.75
1,833837781003059,3.636258e+16,2025-08-01,9.00,8,10,1.00,9.00,8,10,0.89,8.67
2,928039309909868,1.228066e+16,2025-07-01,1.00,1,1,0.00,1.50,1,4,1.22,1.63
3,272463390111355,9.649499e+15,2025-04-01,11.33,2,25,12.10,6.80,0,25,10.57,6.80
4,394042315248610,4.501650e+16,2025-04-01,10.67,0,26,13.61,13.40,0,35,16.12,13.40
5,653345524920819,4.541881e+16,2025-04-01,6.67,1,14,6.66,14.40,1,33,12.60,14.40
6,649115698407618,8.904566e+16,2025-04-01,7.00,6,8,1.00,34.60,6,76,37.80,34.60
7,527218253094718,7.363726e+16,2025-02-01,9.00,5,14,4.58,9.00,5,14,4.58,9.00
8,238236033346518,8.960866e+16,2025-11-01,0.67,0,1,0.58,0.33,0,1,0.52,0.22
9,496199667673826,6.127789e+16,2025-09-01,2.33,0,7,4.04,5.00,0,14,5.93,3.33


<h2> 6. Tabla Tanque Movimientos</h2>

In [15]:
profile_table("silver", "tanque_movimiento", sample_n=10)


📌 PERFIL: silver.tanque_movimiento
Filas: 57753
Columnas: 67


,column_name,column_type,null,key,default,extra
0,num_doc,BIGINT,YES,None,None,None
1,obl17,DOUBLE,YES,None,None,None
2,f_analisis,DATE,YES,None,None,None
3,avg_pago_debito_3m,DOUBLE,YES,None,None,None
4,min_pago_debito_3m,DOUBLE,YES,None,None,None
...,...,...,...,...,...,...
62,stddev_pago_virtual_12m,DOUBLE,YES,None,None,None
63,avg_pago_otros_12m,DOUBLE,YES,None,None,None
64,min_pago_otros_12m,DOUBLE,YES,None,None,None
65,max_pago_otros_12m,DOUBLE,YES,None,None,None



🔎 Nulos (Top):


,column,nulls,null_pct
1,obl17,4.0,0.0001
42,stddev_pago_fisico_9m,0.0,0.0000
48,min_pago_otros_9m,0.0,0.0000
47,avg_pago_otros_9m,0.0,0.0000
46,stddev_pago_virtual_9m,0.0,0.0000
45,max_pago_virtual_9m,0.0,0.0000
44,min_pago_virtual_9m,0.0,0.0000
43,avg_pago_virtual_9m,0.0,0.0000
41,max_pago_fisico_9m,0.0,0.0000
34,stddev_pago_otros_6m,0.0,0.0000



🔑 Duplicados por llave candidata:
- Key: num_doc


,rows_total,distinct_keys,dup_rows
0,57753,33393,24360



📊 Resumen numérico (Top 20 numéricas):


,col,min,avg,max,p50,p95,p99
0,num_doc,5.370803e+09,5.021152e+14,9.999508e+14,4.993825e+14,9.514646e+14,9.906102e+14
1,obl17,-9.353256e+15,4.985394e+16,9.999849e+16,4.982716e+16,9.498879e+16,9.897985e+16
2,avg_pago_debito_3m,0.000000e+00,2.072240e+05,1.526801e+08,0.000000e+00,9.259480e+05,2.864843e+06
3,min_pago_debito_3m,0.000000e+00,1.421329e+05,1.526801e+08,0.000000e+00,6.392162e+05,2.285418e+06
4,max_pago_debito_3m,0.000000e+00,2.756214e+05,1.526801e+08,0.000000e+00,1.240431e+06,3.728689e+06
5,stddev_pago_debito_3m,0.000000e+00,7.704746e+04,7.503774e+07,0.000000e+00,3.521754e+05,1.272566e+06
6,avg_pago_fisico_3m,0.000000e+00,8.087936e+03,1.317920e+08,0.000000e+00,0.000000e+00,0.000000e+00
7,min_pago_fisico_3m,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
8,max_pago_fisico_3m,0.000000e+00,1.957704e+04,2.635840e+08,0.000000e+00,0.000000e+00,0.000000e+00
9,stddev_pago_fisico_3m,0.000000e+00,1.192421e+04,1.863820e+08,0.000000e+00,0.000000e+00,0.000000e+00



👀 Muestra (10 filas):


,num_doc,obl17,f_analisis,avg_pago_debito_3m,min_pago_debito_3m,max_pago_debito_3m,stddev_pago_debito_3m,avg_pago_fisico_3m,min_pago_fisico_3m,max_pago_fisico_3m,...,max_pago_fisico_12m,stddev_pago_fisico_12m,avg_pago_virtual_12m,min_pago_virtual_12m,max_pago_virtual_12m,stddev_pago_virtual_12m,avg_pago_otros_12m,min_pago_otros_12m,max_pago_otros_12m,stddev_pago_otros_12m
0,408768030545892,5.650221e+16,2025-07-01,0.000000e+00,0.00,0.00,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000e+00,0.0,0.00,0.000000e+00,0.0,0.0,0.0,0.0
1,625690886214853,8.505520e+16,2024-12-01,4.484452e+05,448445.22,448445.22,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000e+00,0.0,0.00,0.000000e+00,0.0,0.0,0.0,0.0
2,294344482803237,5.155716e+16,2025-10-01,2.242145e+06,2241792.00,2242850.00,610.836585,0.0,0.0,0.0,...,0.0,0.0,1.447865e+06,0.0,2241792.00,1.103391e+06,0.0,0.0,0.0,0.0
3,196776253827577,4.096992e+15,2024-12-01,0.000000e+00,0.00,0.00,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000e+00,0.0,0.00,0.000000e+00,0.0,0.0,0.0,0.0
4,486596316043962,4.152303e+16,2025-07-01,0.000000e+00,0.00,0.00,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000e+00,0.0,0.00,0.000000e+00,0.0,0.0,0.0,0.0
5,896781401625514,8.549623e+16,2025-07-01,0.000000e+00,0.00,0.00,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000e+00,0.0,0.00,0.000000e+00,0.0,0.0,0.0,0.0
6,324398192071320,5.728579e+16,2025-07-01,3.142569e+05,306385.91,323861.79,8866.029083,0.0,0.0,0.0,...,0.0,0.0,4.808912e+04,0.0,272712.97,9.886199e+04,0.0,0.0,0.0,0.0
7,199946899115743,3.430250e+16,2025-08-01,0.000000e+00,0.00,0.00,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000e+00,0.0,0.00,0.000000e+00,0.0,0.0,0.0,0.0
8,53217655589613,2.210542e+16,2025-08-01,2.547720e+05,254772.00,254772.00,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000e+00,0.0,0.00,0.000000e+00,0.0,0.0,0.0,0.0
9,440254599320892,8.678641e+16,2024-12-01,1.258901e+06,1258901.07,1258901.07,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000e+00,0.0,0.00,0.000000e+00,0.0,0.0,0.0,0.0


<h2> Conclusiones generales </h2>

<h3> 1. silver.clientes</h3>
<p> 1) Tamaño y estructura </p>
<li> Filas: 56,993 </li>
<li> Columnas: 4 </li>
<li> Tabla compacta y probablemente de “población base” para el 360 (cliente/obligación en un corte de fecha).</li>

<p> 2) Calidad de datos (nulos) </p>
<li> obl17 tiene 4 nulos (null_pct 0.0001). </li>
<li> num_doc, f_analisis, var_rta 0 nulos. </li>
<li> calidad de nulos es muy buena. Se eleiminar los registros con oligacion nulos</li>

<li> Trata num_doc y obl17 como STRING/VARCHAR, no como numérico </li>

<h3> 2. silver.canales</h3>
<p> 1) Tamaño y estructura </p>
<li> Filas: 79,098 </li>
<li> Columnas: 66 </li>
<li> La tabla quedó con features tipo: total_mnt_*, max_mnt_*, total_cnt_*, max_cnt_*, share_*, trend_*_ult6_vs_ult12</li>

<p> El proceso realizado con Spark de compactación fue exitoso: se pasa de de +4k columnas a 66, manteniendo métricas de valor analítico (sumas, máximos, shares, tendencias).</p>

<p> 2) Calidad de datos (nulos) </p>
<li> obl17 tiene 4 nulos (null_pct = 0.0001) </li>
<li> calidad de nulos muy alta. </li>
<li> Si el grano es (num_doc,obl17,f_analisis) entonces sin obl17 no hay llave completa → esas filas deben eliminarse</li>

<h3> 3. silver.excedente</h3>
<p> 1) Tamaño y estructura </p>
<li> Filas: 57,753 </li>
<li> Columnas: 35</li>
<li> Variables principales (por nombre): métricas de excedente_pago y porc_pago con ventanas 3m, 6m, 9m, 12m: avg_, min_, max_, stddev_ para cada ventana.
agregaciones por ventana</li>

<p> 2) Calidad de datos (nulos) </p>
<li>  num_doc aparece como BIGINT </li>
<li>  obl17 aparece como DOUBLE </li>
<li>  f_analisis como DATE </li>

<p> Para columnas tipo *_porc_pago_* (especialmente 9m y 12m) hay nulos aprox: ~0.040–0.043 (4%)</p>
<p> obl17 muestra 4 nulos (0.0001)</p>
<p> La tabla es buena en general, pero hay un bloque de features (% pago) con nulos no triviales (~4%). </p>
<p> silver.excedentes tiene 57,753 filas y 35 columnas, con features agregadas por ventanas (3m/6m/9m/12m), útil para modelado.</p>
<p> Existen nulos relevantes (~4%) en variables *_porc_pago_* (especialmente 9m y 12m) → se requiere regla de imputación/flag antes de GOLD. </p>
<p> Se detectan 4 nulos en obl17, lo cual rompe la llave compuesta → revisar y definir tratamiento</p>

<h3> 4. silver.gestiones</h3>
<li> silver.gestiones tiene 57,753 filas y 83 columnas con features por ventanas (3m/6m/9m/12m) tipo avg/min/max/stddev de gestiones, RPC, acuerdos, promesas y rankings.</li>
<li> Nulos muy bajos: solo se observan 4 nulos en obl17, que rompen la llave compuesta y deben tratarse antes de GOLD.</li>
<li> Tipos de llaves inconsistentes (num_doc BIGINT, obl17 DOUBLE) → estandarizar a VARCHAR en SILVER para garantizar joins por (num_doc,obl17,f_analisis).</li>
<li> La distribución muestra alta sparsity (p50=0 en varias métricas): muchos casos sin gestiones, lo cual debe mantenerse como señal y puede convertirse en flags de ausencia.</li>
<li> Se observan outliers en conteos (máximos altos vs mediana 0): considerar clipping/log1p o flags de gestión intensiva en GOLD.</li>


<h3> 5. silver.Moras</h3>
<li> silver.moras tiene 57,753 filas y 12 columnas, con features de mora por ventanas (3m/6m/9m) tipo avg/min/max/stddev.</li>
<li> Nulos casi inexistentes: solo 4 nulos en obl17, lo cual rompe la llave compuesta y debe tratarse antes de GOLD. </li>
<li> Tipos de llaves inconsistentes (num_doc BIGINT, obl17 DOUBLE) → estandarizar llaves a VARCHAR para joins confiables por (num_doc,obl17,f_analisis). </li>
<li> La mora presenta outliers (max muy alto en algunas ventanas), por lo que en GOLD se recomiendan flags de mora alta/extrema y evaluar clipping/logs según impacto.</li>

<h3> 6. silver.Tanque Movimientos</h3>
<li> silver.tanque_movimiento tiene 57,753 filas y 67 columnas con features de pagos por canal (débito/físico/virtual/otros) y ventanas (3m/6m/9m/12m) usando avg/min/max/stddev.</li>
<li> Nulos casi inexistentes: solo 4 nulos en obl17 → rompen la llave compuesta y deben tratarse antes de GOLD.</li>
<li> Tipos de llaves inconsistentes (num_doc BIGINT, obl17 DOUBLE) → estandarizar a VARCHAR para joins confiables por (num_doc,obl17,f_analisis). </li>
<li> Alta sparsity (p50=0 en varias métricas): muchos casos sin pagos en ciertas ventanas/canales; mantener como señal y crear flags de ausencia. </li>
<li> Outliers/skew en montos de pagos (p95/p99 altos): en GOLD se recomienda log1p y/o clipping + flags de pago alto.</li>

In [16]:
con.close()